https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/120.0.6099.109/win64/chromedriver-win64.zip

In [1]:
#imports here
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
import time

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-notifications')
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')

In [ ]:
driver = webdriver.Remote(
    command_executor="http://localhost:4444/wd/hub", options=options
)

In [3]:
service = Service(executable_path='../webdriver/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)

In [4]:
import os
from dotenv import load_dotenv

EMAIL = None
PASS = None
MAIN_GROUP_ID = None

EMAIL = os.environ.get("EMAIL")
PASS = os.environ.get("PASS")
MAIN_GROUP_ID = os.environ.get("MAIN_GROUP_ID")
if load_dotenv():
    EMAIL = os.getenv("EMAIL")
    PASS = os.getenv("PASS")
    MAIN_GROUP_ID = os.getenv("MAIN_GROUP_ID")
    
if not EMAIL or not PASS or not MAIN_GROUP_ID:
    print("Environment not found. Please check if the env has EMAIL, PASS and MAIN_GROUP_ID.")

In [5]:
driver.get("https://mbasic.facebook.com/login.php")

username = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']"))
)
password = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']"))
)

# enter username and password
username.clear()
username.send_keys(EMAIL)
password.clear()
password.send_keys(PASS)
WebDriverWait(driver, 2).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[type="submit"]'))
).click()
WebDriverWait(driver, 2).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'tr > td > div > form[method="post"] + div a'))
).click()

In [12]:
box_comments = driver.find_elements(
    By.CSS_SELECTOR,
    "#m_story_permalink_view > div[id] > div > div:not([id]) > div:has(div)",
)
for box in box_comments:
    try:
        print(
            box.find_element(
                By.CSS_SELECTOR, "div > div:nth-of-type(4) > div[id] > div a[href]"
            ).get_attribute("href")
        )
    except:
        print(None)

https://mbasic.facebook.com/comment/replies/?ctoken=1502731976965957_1503661446873010&count=1&curr&pc=1&ft_ent_identifier=1502731976965957&eav=Afby3RimpmBuv3Yd3Lb8-5u4ixqUaSWBtRroN4IvBIfzAyvZ2eynOBFM2j9nglnklWc&gfid=AQAv90t2F3vY3OeaADE&paipv=0&refid=18&__tn__=R
https://mbasic.facebook.com/comment/replies/?ctoken=1502731976965957_1503362000236288&count=2&curr&pc=1&ft_ent_identifier=1502731976965957&eav=AfZq1yW7ZIyLHIXOEoieg-9cWkIF-obGVA7xTSYz5BVU83WvoFW1E_HgajfN93VvtkM&gfid=AQBVc1sZIVJFWGD2wYQ&paipv=0&refid=18&__tn__=R
https://mbasic.facebook.com/comment/replies/?ctoken=1502731976965957_1502858700286618&count=3&curr&pc=1&ft_ent_identifier=1502731976965957&eav=Afb4Iu7ffL_FP5zuzfVjHd9aU0gZdPt_rGrmxIqJ11Ab0J1WxBrpkqI06HtwgsLkpP4&gfid=AQBgl4nxPTQjJw9dUjk&paipv=0&refid=18&__tn__=R
None
https://mbasic.facebook.com/comment/replies/?ctoken=1502731976965957_1503775143528307&count=1&curr&pc=1&ft_ent_identifier=1502731976965957&eav=AfZyhRC0vXTDAf9IOw1Coy5HJBM9LQ5mOSQ_rbBwEHOFxS0n65tffDMd5kkSIaK77p

In [13]:
import re
import sys

def replies_scraping(box_comment):
    try:
        replies_url = box_comment.find_element(
            By.CSS_SELECTOR, "div > div:nth-of-type(4) > div[id] > div a[href]"
        ).get_attribute("href")

        driver.execute_script(f"window.open('{replies_url}', '_blank')")
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[2])
    except:
        return []

    try:
        you_is_block = driver.find_element(
            By.CSS_SELECTOR,
            'div[title="You’re Temporarily Blocked"] > h2',
        ).text

        print("You’re Temporarily Blocked from viewing replies.")
        sys.exit(1)
    except:
        None


    reply_id = re.search(r"ctoken=(\d+_\d+)", driver.current_url).group(1)

    replies = []


    while True:
        box_replies = driver.find_elements(
            By.CSS_SELECTOR,
            'div[id="root"] > div[class] > div[id] + div > div:has(div)',
        )

        for idx, box in enumerate(box_replies):
            reply = dict()
            reply["reply_by"] = box.find_element(By.CSS_SELECTOR, "div > h3").text
            try:
                reply["reply_to"] = box.find_element(
                    By.CSS_SELECTOR, "div > div:nth-child(2) > a"
                ).text
            except:
                None

            reply_spans = box.find_elements(
                By.CSS_SELECTOR, "div > div:nth-child(2) > span"
            )
            reply["reply"] = "".join([reply_span.text for reply_span in reply_spans])
            reply["reply_order"] = idx
            print(
                f"{reply['reply_by']} { 'To' + reply.get('reply_to', None) if reply.get('reply_to', None) else None } -> reply: {reply['reply']}"
            )
            replies.append(reply)
            
        try:
            prev_comment_btn = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable(
                    (
                        By.CSS_SELECTOR,
                        f'div[id="root"] > div[class] > div + div:nth-of-type(2) > div[id="comment_replies_more_1:{reply_id}"] > a',
                    )
                )
            )
            prev_comment_btn.click()
        except:
            break
        
    driver.close()
    time.sleep(1)
    driver.switch_to.window(driver.window_handles[1])
    return replies

In [16]:
import re

def comments_scraping():
    post_id = re.search(r"/permalink/(\d+)", driver.current_url).group(1)
    comments = []
    next_page_btn_id = None

    while True:
        box_comments = driver.find_elements(
            By.CSS_SELECTOR,
            "#m_story_permalink_view > div[id] > div > div:not([id]) > div:has(div)",
        )
        if len(box_comments) == 0:
            return []
        for box_comment in box_comments:
            comment = dict()
            comment["comment_by"] = box_comment.find_element(
                By.CSS_SELECTOR, "div > h3"
            ).text
            comment["comment"] = box_comment.find_element(
                By.CSS_SELECTOR, "div > div:nth-child(2)"
            ).text
            print(f"{comment['comment_by']} -> comment: {comment['comment']}")
            driver.switch_to.window(driver.window_handles[1])

            replies = replies_scraping(box_comment)
            if len(replies) != 0:
                comment["replies"] = replies

            comments.append(comment)

        if next_page_btn_id is None:
            try: 
                next_page_btn_id = driver.find_element(
                    By.CSS_SELECTOR,
                    f'#m_story_permalink_view div:last-of-type > div > div:nth-of-type(5) > div:has(>a)',
                ).get_attribute('id')
            except:
                break
        try:
            next_page_btn = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable(
                    (
                        By.CSS_SELECTOR,
                        f'div[id="root"] div#{next_page_btn_id} > a',
                    )
                )
            )
            next_page_btn.click()
        except:
            break
    print(f"Complete post_id: {post_id}")
    return comments

In [15]:
import re

def post_scraping():
    post_id = re.search(r"/permalink/(\d+)", driver.current_url).group(1)
    post = dict()
    post["post_id"] = post_id
    post["topic"] = driver.find_element(
        By.CSS_SELECTOR,
        '#m_story_permalink_view div:not([id]) > div > div > div',
    ).text
    post["topic_by"] = driver.find_element(
        By.CSS_SELECTOR, "td > header > h3 > span > strong:first-child > a"
    ).text
    post["comments"] = comments_scraping()
    return post

In [17]:
driver.switch_to.window(driver.window_handles[0])

main_group_link = f"https://mbasic.facebook.com/groups/{MAIN_GROUP_ID}/"
driver.get(main_group_link)

In [18]:
def reset_tab():
    for idx, tab in enumerate(driver.window_handles):
        if idx == 0:
            continue
        driver.switch_to.window(tab)
        driver.close()
    driver.switch_to.window(driver.window_handles[0])
    driver.get(main_group_link)

In [20]:
reset_tab()

In [21]:
import sqlite3

while True:
    anchor_all = driver.find_elements( #The program requires a discussion thread.
        By.CSS_SELECTOR,
        "#m_group_stories_container article > footer > div:last-child > a:nth-of-type(3)",
    )
    anchor_all = [a.get_attribute("href") for a in anchor_all]
    anchor_post_shares = driver.find_elements(
        By.CSS_SELECTOR,
        "article:has(article) > footer > div:last-child > a:nth-of-type(3)",
    )
    anchor_post_shares = [a.get_attribute("href") for a in anchor_post_shares]
    anchors = list(set(anchor_all) - set(anchor_post_shares))

    for a in anchors:
        post_id = re.search(r"/permalink/(\d+)", a).group(1)
        with sqlite3.connect("posts.db") as connection:
            cursor = connection.cursor()
            rows = cursor.execute(
                "SELECT post_id FROM posts WHERE post_id=?", (post_id,)
            ).fetchall()
            if len(rows) != 0:
                print(f"Already have post_id : {post_id}")
                continue
        driver.execute_script(f"window.open('{a}', '_blank')")
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[1])
        print(f"Start scraping post_id: {post_id}.")

        post = post_scraping()

        with sqlite3.connect("posts.db") as connection:
            cursor = connection.cursor()
            cursor.execute(
                "INSERT INTO posts (post_id, topic_by, topic) VALUES (?, ?, ?)",
                (post["post_id"], post["topic_by"], post["topic"]),
            )
            post_id1 = cursor.lastrowid
            comments = post.get("comments", [])
            if len(comments) == 0:
                continue
            for comment in comments:
                cursor.execute(
                    "INSERT INTO comments (comment_by, comment, post_id) VALUES (?, ?, ?)",
                    (comment["comment_by"], comment["comment"], post_id1),
                )
                comment_id = cursor.lastrowid
                replies = comment.get("replies", [])
                if len(replies) == 0:
                    continue
                for reply in replies:
                    cursor.execute(
                        "INSERT INTO replies (reply_by, reply_to, reply, reply_order, comment_id) VALUES (?, ?, ?, ?, ?)",
                        (reply["reply_by"], reply.get("reply_to", None), reply["reply"], reply['reply_order'], comment_id),
                    )
            connection.commit()
        print(f"Save to database complated as post_id: {post_id}.")

        driver.close()
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)
    try:
        see_more_posts_btn = WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable(
                (
                    By.CSS_SELECTOR,
                    "section + div > a:has(span)",
                )
            )
        )
        see_more_posts_btn.click()
    except:
        print("An error occurred.")
        print(f"Error URL: {driver.current_url}")
        reset_tab()


Start scraping post_id: 1503901333515688.
Natsu -> comment: Mindset เล่มนี้เลยค่า ✨
🧡 https://shope.ee/5fP9YQ2lwi
Natsu Tohttps://shope.ee/qIlQGrC8H -> reply: พัฒนาศักยภาพลอง
Pete Phitthawat -> comment: https://shope.ee/1VZac14emM
ด้านการพูดแนะนำเล่มนี้เลยครับ การพูดเป็นสกิลทักษะอีกอย่างที่ถ้าเราพูดได้ดี ใช้คำในสถานการณ์ต่างๆเป็น ช่วยเพิ่มโอกาสในชีวิตได้ดีมากๆครับ
Julajakra Tri Washirapong -> comment: เพียบเลยครับ เอากี่เล่มดี
Complete post_id: 1503901333515688
Save to database complated as post_id: 1503901333515688.
Start scraping post_id: 1503433253562496.
Chanita Butrattana -> comment: ผู้พูด รู้สึกอย่างไร 🤭
Rattana Sawangwong -> comment: เฉยๆค่ะคำนี้ แต่เราคิดเสมอว่าถ่อยไม่ได้แปลว่าเท่ หยาบคายไม่ได้แปลว่าเป็นคนตรงค่ะ
หนอน บุ้ง -> comment: ไม่ใช่จริตของเรานะคะ เเต่ก็เเล้วเเต่ใครสดวกเเบบไหนอะค่ะ
ปล.ไม่ชอบเสียเวลากับอะไรที่มันไม่ได้ประโยชน์ ชนะไปก็ไม่เห็นได้อะไร😅
Joy Rattikan -> comment: ชอบค่ะ😆
Mack Sukrit -> comment: คิดยังไงหรอครับ คิดว่านักปราชญ์ ที่มากไปด้วยความรู้ของหนังสือจะไม่

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.129)
Stacktrace:
	GetHandleVerifier [0x00007FF7F8462142+3514994]
	(No symbol) [0x00007FF7F8080CE2]
	(No symbol) [0x00007FF7F7F276AA]
	(No symbol) [0x00007FF7F7F00AFD]
	(No symbol) [0x00007FF7F7F9CB1B]
	(No symbol) [0x00007FF7F7FB218F]
	(No symbol) [0x00007FF7F7F95D93]
	(No symbol) [0x00007FF7F7F64BDC]
	(No symbol) [0x00007FF7F7F65C64]
	GetHandleVerifier [0x00007FF7F848E16B+3695259]
	GetHandleVerifier [0x00007FF7F84E6737+4057191]
	GetHandleVerifier [0x00007FF7F84DE4E3+4023827]
	GetHandleVerifier [0x00007FF7F81B04F9+689705]
	(No symbol) [0x00007FF7F808C048]
	(No symbol) [0x00007FF7F8088044]
	(No symbol) [0x00007FF7F80881C9]
	(No symbol) [0x00007FF7F80788C4]
	BaseThreadInitThunk [0x00007FFA8D5C257D+29]
	RtlUserThreadStart [0x00007FFA8DEAAA58+40]


# ไม่เกี่ยว

In [47]:
import sqlite3

with sqlite3.connect("posts.db") as connection:
    # Create a cursor within the 'with' block
    cursor = connection.cursor()

    # Delete all records from the 'posts' table
    cursor.execute("DELETE FROM posts")

    # Commit the changes
    connection.commit()

In [20]:
import sqlite3

with sqlite3.connect("posts.db") as connection:
    cursor = connection.cursor()
    cursor.execute(
        """
      CREATE TABLE IF NOT EXISTS posts (
          id INTEGER PRIMARY KEY,
          post_id INTEGER NOT NULL,
          topic_by TEXT,
          topic TEXT
      )
  """
    )

    cursor.execute(
        """
      CREATE TABLE IF NOT EXISTS comments (
          id INTEGER PRIMARY KEY,
          comment_by TEXT NOT NULL,
          comment TEXT NOT NULL,
          post_id INTEGER NOT NULL,
          FOREIGN KEY (post_id) REFERENCES posts(id)
      )
  """
    )

    cursor.execute(
        """
      CREATE TABLE IF NOT EXISTS replies (
          id INTEGER PRIMARY KEY,
          reply_by TEXT NOT NULL,
          reply_to TEXT,
          reply TEXT NOT NULL,
          reply_order INTEGER NOT NULL,
          comment_id INTEGER NOT NULL,
          FOREIGN KEY (comment_id) REFERENCES comments(id)
      )
  """
    )

In [ ]:
import pyperclip
pyperclip.copy(anchors[0])